# Travel Planning Agents with Google Gemini API

This notebook demonstrates multi-agent collaboration using AutoGen with Google's Gemini API.

In [1]:
# Import required libraries
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
import os

In [2]:
# Configure Google Gemini API client
# Make sure to set your GOOGLE_API_KEY environment variable
# You can get an API key from: https://makersuite.google.com/app/apikey

model_client = OpenAIChatCompletionClient(
    model="gemini-2.5-pro",
    api_key=os.getenv("GOOGLE_API_KEY"),  # use GOOGLE_API_KEY env var
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)


In [3]:
# Define the travel planning agents

planner_agent = AssistantAgent(
    "planner_agent",
    model_client=model_client,
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request.",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=model_client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided.",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=model_client,
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=model_client,
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed final travel plan. You must ensure that the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.",
)

In [4]:
# Create a termination condition
termination = TextMentionTermination("TERMINATE")

# Create the team with RoundRobin group chat
team = RoundRobinGroupChat(
    participants=[planner_agent, local_agent, language_agent, travel_summary_agent],
    termination_condition=termination,
)

In [5]:
# Run the team with a travel planning task
import asyncio

async def run_travel_planning():
    result = await Console(
        team.run_stream(
            task="Plan a 5-day trip to Mexico City. Include must-see attractions, local experiences, and language tips."
        )
    )
    return result

# Run the async function
await run_travel_planning()

---------- TextMessage (user) ----------
Plan a 5-day trip to Mexico City. Include must-see attractions, local experiences, and language tips.


Error processing publish message for planner_agent_1a080af5-1cde-492b-92aa-1855cdbf2145/1a080af5-1cde-492b-92aa-1855cdbf2145
Traceback (most recent call last):
  File "/workspaces/Python-Autogen/.venv/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/Python-Autogen/.venv/lib/python3.12/site-packages/autogen_core/_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/Python-Autogen/.venv/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/Python-Autogen/.venv/lib/python3.12/site-packages/autogen_core/_routed_agent.py", line 485, in on_messag

RuntimeError: NotFoundError: Error code: 404 - [{'error': {'code': 404, 'message': 'models/gemini-1.5-pro is not found for API version v1main, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.', 'status': 'NOT_FOUND'}}]
Traceback:
Traceback (most recent call last):

  File "/workspaces/Python-Autogen/.venv/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_chat_agent_container.py", line 133, in handle_request
    async for msg in self._agent.on_messages_stream(self._message_buffer, ctx.cancellation_token):

  File "/workspaces/Python-Autogen/.venv/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py", line 953, in on_messages_stream
    async for inference_output in self._call_llm(

  File "/workspaces/Python-Autogen/.venv/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py", line 1109, in _call_llm
    model_result = await model_client.create(
             